# Análisis y traducción de tweets de texto común a ASCII

Notebook generadora del dataset para utilizar en el análisis. Toma la base de datos, remueve las palabras y signos a ignorar, traduce los emojis a ASCII y guarda el dataset final.

In [41]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from TweetAnalysis import hermes

#### Cargo la base de datos clasificada y genero las variables de importancia

In [42]:
TW = pd.read_csv('base_corr.csv')

Estos son los tweets no clasificados

In [43]:
DATA = pd.read_csv('tweets.csv',sep=';')

MSJ contiene al tweet, Tag si es meteorológico o no, Num el id del tweet, Who el nombre de quien los clasificó y User el id de quien twiteó (eso lo tuvimos que sacar de la base de datos original).

In [44]:
MSJ = TW['Twit']
TAG = TW['Tag']
NUM = TW['Num']
WHO = TW['Nombre']
USER = DATA['user_id'][np.asarray(NUM)]

In [45]:
FINAL = pd.DataFrame(list(zip(MSJ, TAG, NUM, USER, WHO)), columns = ['MSJ', 'TAG', 'NUN', 'USER', 'WHO'])

## La función Hermes: normaliza los tweets y borra el spam meteorológico

In [46]:
FINAL = hermes(FINAL, 'MSJ', 'USER', 5)

## Corrector ortográfico

In [70]:
FINAL['msj_ort'] = np.full((FINAL.shape[0], 1), 999)

print('\033[1m Estás seguro de querer restaurar el archivo BaseOrt.csv? \n Se perderán todas las correcciones ortográficas realizadas hasta ahora \n Responder con Siii en caso afirmativo \033[0m \n')
desc = input()
if desc == 'Siii':
    FINAL.to_csv(r'BaseOrt.csv', index = None, sep = ';')
else: print('\n Ok, la base de datos corregida ortográficamente no se restauró.\n')

 Estás seguro de querer restaurar el archivo BaseOrt.csv? 
 Se perderán todas las correcciones ortográficas realizadas hasta ahora 
 Responder con Siii en caso afirmativo  

Siii


Hago esta vuelta para no corregir todo cada vez que corro el script, sino que se sube el archivo hasta donde se lo corrigió

In [71]:
ORT = pd.read_csv('BaseOrt.csv', sep = ';')
ORT

,MSJ,TAG,NUN,USER,WHO,msj_norm,msj_ort
0,Esta lloviendo,1,NaN,NaN,NaN,esta lloviendo,999
1,quiero ir al centro a la mañana pero seguro ll...,0,41082.0,1.027416e+18,NaN,quiero ir centro mañana pero seguro llueve,999
2,"@paloma_CAI7 Si, llueve y mucho !! 😒",1,86721.0,9.704968e+17,NaN,si llueve mucho \U0001f612,999
3,@rozalazar9 Si llueve les hago 🤗,0,107171.0,1.708397e+09,NaN,si llueve hago \U0001f917,999
4,Hasta cuándo le voy a seguir teniendo miedo a ...,0,57648.0,8.477091e+17,NaN,hasta cuando voy seguir teniendo miedo torment...,999
5,"@Eze2450 Sea la estación que sea , no hay plat...",1,67700.0,1.912568e+08,NaN,sea estacion sea no hay plata ahora llueve...,999
6,Llueve y no tengo Internet...me espera tremend...,1,68738.0,2.907214e+09,NaN,llueve no tengo internet espera tremenda tarde,999
7,Que linda noche de lluvia 💖,1,41774.0,2.266866e+09,NaN,que linda noche lluvia \U0001f496,999
8,"Miércoles de mucho calor, previo a las lluvias...",1,42346.0,1.737797e+08,NaN,miercoles mucho calor previo lluvias,999
9,Me gusta cuando llueve porque parece que se es...,0,17694.0,2.560815e+09,NaN,me gusta cuando llueve porque parece estan fri...,999


In [49]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('listado-general.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return list(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return list(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [84]:
for i in range(ORT.shape[0]):
    if ORT['msj_ort'][i] == 999:
        txt = ORT['msj_norm'][i].split()
        for j in txt:
            A = candidates(j)
            if len(A)>1: print(j, A)
    else: print('Checked!')

esta ['eta', 'esa', 'seta', 'asta', 'osta', 'enta', 'esca', 'este', 'cesta', 'festa', 'gesta', 'mesta', 'resta', 'testa', 'estar', 'estay']
lloviendo ['llovido', 'llovido']
quiero ['quier', 'quieto', 'quijero', 'quiebro']
llueve ['leve', 'nueve', 'lieve', 'leve', 'nueve', 'lieve', 'leve', 'leve', 'aleve', 'lieve', 'llave', 'lleva', 'llave', 'aleve', 'nueve', 'lieve', 'nueve', 'llave', 'lieve', 'lieve', 'llave', 'llueca', 'lleva', 'llueca']
llueve ['leve', 'nueve', 'lieve', 'leve', 'nueve', 'lieve', 'leve', 'leve', 'aleve', 'lieve', 'llave', 'lleva', 'llave', 'aleve', 'nueve', 'lieve', 'nueve', 'llave', 'lieve', 'lieve', 'llave', 'llueca', 'lleva', 'llueca']
llueve ['leve', 'nueve', 'lieve', 'leve', 'nueve', 'lieve', 'leve', 'leve', 'aleve', 'lieve', 'llave', 'lleva', 'llave', 'aleve', 'nueve', 'lieve', 'nueve', 'llave', 'lieve', 'lieve', 'llave', 'llueca', 'lleva', 'llueca']
hago ['hao', 'bago', 'fago', 'gago', 'lago', 'mago', 'pago', 'sago', 'vago', 'higo', 'hogo', 'hado', 'halo', 'ha

battle ['atole', 'bate', 'batel', 'bable', 'bafle', 'baile', 'bate', 'batel', 'bable', 'bafle', 'baile', 'bate', 'bate', 'baste', 'batel', 'batel', 'batel', 'bable', 'bafle', 'baile', 'balele', 'baste', 'bable', 'batel', 'balele', 'bafle', 'baile', 'atole']
royale ['roble', 'roble', 'roya', 'roya', 'boyal', 'poyal', 'rodal', 'rojal', 'rosal', 'rotal', 'boyal', 'poyal', 'roble', 'robalo', 'rodal', 'rodaje', 'rojal', 'ropaje', 'rosal', 'rotal', 'rozable', 'roble', 'rodaje', 'ropaje', 'robalo', 'rozable']
duerme ['verme', 'verme', 'fuerte', 'inerme', 'muerte', 'muermo', 'quermes', 'suerte', 'tuerce', 'verme', 'inerme', 'verme', 'duarte', 'duende', 'tuerce', 'duende', 'duerna', 'duerno', 'fuerte', 'muerte', 'suerte', 'duarte', 'duerna', 'muermo', 'duerno', 'quermes']
ahre ['ahe', 'acre', 'agre', 'aire', 'arre']
abran ['bran', 'aran', 'abra']
datos ['dato', 'catos']
aaaaay ['ajajay', 'ananay', 'atatay', 'ayayay', 'ajajay', 'ananay', 'atatay', 'ayayay']
boludo ['boldo', 'coludo', 'bocudo', '

cagarnos ['cavaros', 'calainos', 'cavaros', 'calainos']
hacen ['haren', 'hacan', 'hacer']
sienta ['siena', 'tienta', 'siesta', 'sienita']
estoy ['estay', 'estol', 'estor', 'estos']
parta ['arta', 'pata', 'para', 'carta', 'marta', 'sarta', 'tarta', 'perta', 'porta', 'palta', 'pasta', 'pauta', 'parca', 'paria', 'parla', 'parra', 'parva', 'parte', 'parto', 'parata']
llueve ['leve', 'nueve', 'lieve', 'leve', 'nueve', 'lieve', 'leve', 'leve', 'aleve', 'lieve', 'llave', 'lleva', 'llave', 'aleve', 'nueve', 'lieve', 'nueve', 'llave', 'lieve', 'lieve', 'llave', 'llueca', 'lleva', 'llueca']
saben ['saten', 'sabeo', 'saber']
estoy ['estay', 'estol', 'estor', 'estos']
yendo ['vendo', 'zendo']
caida ['cada', 'caid', 'baida', 'vaida', 'zaida', 'crida', 'cuida', 'calda', 'carda', 'cauda', 'caima', 'caido', 'casida']
tranquila ['tranquil', 'trasquila', 'tranquiza', 'tranquilo', 'tranquilla', 'tranquilla', 'tranquilar']
desmayarse ['desalarse', 'desmayar', 'desmayar', 'descararse', 'desalarse', 'descar

quiero ['quier', 'quieto', 'quijero', 'quiebro']
llueva ['lleva', 'llueca']
llueve ['leve', 'nueve', 'lieve', 'leve', 'nueve', 'lieve', 'leve', 'leve', 'aleve', 'lieve', 'llave', 'lleva', 'llave', 'aleve', 'nueve', 'lieve', 'nueve', 'llave', 'lieve', 'lieve', 'llave', 'llueca', 'lleva', 'llueca']
lluvias ['lluvia', 'lluvial']
generaron ['generar', 'generar', 'generacion', 'generador', 'generacion', 'generador', 'generacion', 'generacion']
antonio ['antonimo', 'antonino']
ingresa ['ingreso', 'ingresar']
mucha ['cucha', 'ducha', 'hucha', 'lucha', 'pucha', 'macha', 'mecha', 'micha', 'mocha', 'mucho']
rios ['ros', 'rio', 'riso', 'dios']
dias ['dia', 'dios', 'dial']
recomienda ['comienda', 'encomienda', 'comienda', 'recomendar', 'encomienda', 'encomienda', 'encomienda', 'encomienda', 'remolienda', 'remolienda', 'recompensa', 'recompensa', 'encomienda', 'recomendar']
ordenes ['modenes', 'orense', 'oranes', 'lorenes', 'oranes', 'orden', 'orden', 'modenes', 'orense', 'modenes', 'modenes', 'mod

mojarte ['marte', 'marte', 'moare', 'mojete', 'mogate', 'mojete', 'moare', 'mojar', 'mojar', 'bocarte', 'majarete', 'bocarte', 'mogate', 'mojete', 'mojete', 'mojarra', 'mojarra', 'majarete']
tengo ['rengo', 'tango', 'tongo', 'tenso']
durmiendo ['hurmiento', 'hurmiento', 'durmiente', 'durmiente']
estoy ['estay', 'estol', 'estor', 'estos']
comieron ['omicron', 'coiron', 'comedon', 'comezon', 'coiron', 'comilon', 'comilon', 'coimero', 'comidero', 'cominero', 'coimero', 'cocheron', 'colleron', 'comadron', 'cocheron', 'colleron', 'omicron', 'comadron', 'comilon', 'comision', 'comedon', 'comision', 'comilon', 'comezon', 'comidero', 'cominero']
inversiones ['inversion', 'inversion']
años ['año', 'amos', 'apos', 'añas', 'añoso']
controlamos ['controlador', 'controlador']
etc ['et', 'eta']
nuestra ['muestra', 'nuestro']
dolares ['dares', 'dares', 'dolar', 'dolar', 'dolaje', 'dolame', 'tomares', 'dogaresa', 'tomares', 'dolaje', 'dolame', 'dogaresa']
maneja ['manea', 'madeja', 'manija', 'manera',

...

max ['ax', 'fax', 'mal', 'man', 'mar', 'mas']
precipitaciones ['precipitacion', 'precipitacion']
tormentas ['tormenta', 'tormentar']
cambiando ['cambado', 'cambado', 'cambiazo', 'cambiada', 'cambiador', 'cambiazo', 'cambriano', 'combinado', 'combinado', 'cambiario', 'cambiazo', 'cambiario', 'cambiante', 'cambiazo', 'cambiada', 'cambiante', 'cambriano', 'cambiador']
mirando ['mirado', 'miranda']
puede ['sede', 'sede', 'pece', 'peje', 'pene', 'pepe', 'pedo', 'pelde', 'pure', 'pudu', 'pure', 'pues', 'pues', 'leude', 'pelde', 'cuete', 'duende', 'leude', 'nueve', 'queda', 'quedo', 'rueda', 'ruedo', 'sede', 'pelde', 'pierde', 'pelde', 'plebe', 'preda', 'sede', 'puado', 'pelde', 'pure', 'puzle', 'plebe', 'pueble', 'pece', 'peje', 'puzle', 'pueble', 'pene', 'puente', 'pepe', 'pure', 'pues', 'cuete', 'puente', 'nueve', 'queda', 'rueda', 'preda', 'pedo', 'quedo', 'ruedo', 'puado', 'pues', 'pudu', 'pueble', 'pelde', 'duende', 'puente', 'pierde', 'pueble', 'puente']
quiero ['quier', 'quieto', 'qui

quiero ['quier', 'quieto', 'quijero', 'quiebro']
bañarme ['adarme', 'añazme', 'bañar', 'bañar', 'adarme', 'bayarte', 'añazme', 'bayarte']
min ['in', 'mi', 'din', 'fin', 'gin', 'hin', 'nin', 'pin', 'rin', 'sin', 'yin', 'man', 'mia', 'mil', 'mio', 'miz', 'amin', 'mina', 'mino']
bondi ['onda', 'onde', 'boli', 'boda', 'bode', 'body', 'bon', 'moni', 'poni', 'boli', 'bona', 'bono', 'borni', 'bon', 'dond', 'bona', 'bono', 'borni', 'candi', 'conde', 'condir', 'dond', 'dandi', 'donde', 'fendi', 'fonda', 'fondo', 'hindi', 'honda', 'hondo', 'jondo', 'moni', 'mandi', 'monfi', 'monda', 'mondo', 'poni', 'ponci', 'ronda', 'rondo', 'rondin', 'rondis', 'rondiz', 'sonda', 'tondo', 'tondiz', 'vendi', 'zonda', 'candi', 'dandi', 'mandi', 'banda', 'bando', 'bandin', 'bandir', 'fendi', 'vendi', 'hindi', 'boli', 'boldo', 'boqui', 'borni', 'borda', 'borde', 'bordo', 'bona', 'bonsai', 'bonasi', 'ponci', 'monfi', 'bonga', 'bongo', 'boli', 'borni', 'bono', 'borni', 'bonasi', 'bonsai', 'boqui', 'bonzo', 'onda', 'b

dicen ['cien', 'din', 'bien', 'cien', 'sien', 'diez', 'din', 'fice', 'doce', 'dije', 'cien', 'arcen', 'bien', 'cien', 'fice', 'himen', 'limen', 'liceo', 'micer', 'niceno', 'piden', 'picon', 'picea', 'piceo', 'sien', 'decena', 'deceno', 'doce', 'docena', 'doceno', 'arcen', 'diasen', 'piden', 'dije', 'dijes', 'himen', 'limen', 'dimes', 'dison', 'diasen', 'divan', 'divan', 'dicaz', 'dicha', 'dicho', 'picon', 'dison', 'diucon', 'picea', 'dicha', 'liceo', 'piceo', 'dicho', 'dirceo', 'micer', 'dijes', 'dimes', 'diez', 'dicaz', 'diasen', 'dirceo', 'diucon', 'diasen', 'decena', 'docena', 'dicente', 'niceno', 'deceno', 'doceno', 'dicente']
sea ['ea', 'se', 'esa', 'cea', 'dea', 'gea', 'jea', 'mea', 'nea', 'pea', 'rea', 'tea', 'sed', 'sel', 'sen', 'seo', 'ser', 'ses', 'set', 'seca', 'seda', 'sega', 'sema', 'sena', 'sera', 'seta']
patitas ['patas', 'patas', 'patas', 'patas', 'parias', 'patito', 'catita', 'patata', 'pateta', 'patota', 'patina', 'patito', 'catita', 'latitar', 'parias', 'patata', 'pa

durmiendo ['hurmiento', 'hurmiento', 'durmiente', 'durmiente']
dio ['io', 'do', 'di', 'ido', 'cio', 'lio', 'mio', 'pio', 'rio', 'tio', 'duo', 'dia', 'dij', 'din', 'dix', 'odio', 'dino', 'divo', 'dios']
preocupen ['preocupar', 'preocupar']
estoy ['estay', 'estol', 'estor', 'estos']
tengo ['rengo', 'tango', 'tongo', 'tenso']
estoy ['estay', 'estol', 'estor', 'estos']
cago ['cao', 'bago', 'fago', 'gago', 'lago', 'mago', 'pago', 'sago', 'vago', 'cabo', 'caco', 'cado', 'cajo', 'calo', 'cano', 'capo', 'caro', 'caso', 'cato', 'cavo', 'cayo', 'cazo', 'cargo', 'cagon']
llueve ['leve', 'nueve', 'lieve', 'leve', 'nueve', 'lieve', 'leve', 'leve', 'aleve', 'lieve', 'llave', 'lleva', 'llave', 'aleve', 'nueve', 'lieve', 'nueve', 'llave', 'lieve', 'lieve', 'llave', 'llueca', 'lleva', 'llueca']
hay ['ay', 'ha', 'cay', 'gay', 'lay', 'hoy', 'huy', 'hao', 'haz', 'haya', 'hayo']
caniche ['janiche', 'caliche', 'canicie']
una ['na', 'un', 'ana', 'dna', 'upa', 'ura', 'uta', 'uva', 'uno', 'buna', 'cuna', 'duna

KeyboardInterrupt: 

In [77]:
txt=ORT['msj_norm'][0].split()

In [78]:
txt[0]

'esta'

In [82]:
candidates('jola')

['ola',
 'bola',
 'cola',
 'dola',
 'gola',
 'hola',
 'mola',
 'rola',
 'tola',
 'yola',
 'jora',
 'josa',
 'jota',
 'joya']

In [85]:
FINAL['msj_norm'].to_csv(r'BaseOrt.csv', index = None, sep = ';')